In [12]:
import pandas as pd
import numpy as np
import glob
import os
import hashlib
import matplotlib.pyplot as plt
import seaborn as sns

Collecting CSV files

In [2]:
folder = "C:/Users/HP/Desktop/Traineeship/data_istat/values"

# Grab all CSV files
all_files = glob.glob(os.path.join(folder, "*.csv"))

Assigning a unique ID for each listing

In [3]:
# Define columns that should uniquely identify a listing 
id_columns = [
    'Prov', 'Comune_ISTAT', 'Comune_descrizione', 'Fascia', 'Zona', 'Descr_Tipologia', 'Stato'
]

dfs = []

for f in all_files:
    # Extract filename without extension
    filename = os.path.splitext(os.path.basename(f))[0]
    
    # Extract semester code
    parts = filename.split("_")
    semester_code = parts[-2]  # second to last part
    year = semester_code[:4]
    sem = semester_code[4]
    semester = f"{year}_S{sem}"
    
    # Read CSV, skip first title line
    df = pd.read_csv(f, sep=';', skiprows=1)
    
    # Strip whitespace and remove BOM from column names
    df.columns = df.columns.str.strip().str.replace('\ufeff','')
    
    # Hash concatenated string
    df['listing_id'] = pd.util.hash_pandas_object(df[id_columns].fillna(''), index=False).astype(str)

    # Add semester column
    df['semester'] = semester
    
    # Convert numeric columns to numeric type
    numeric_cols = ['Compr_min', 'Compr_max', 'Loc_min', 'Loc_max']

    for col in numeric_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.', regex=False), errors='coerce')
    
    dfs.append(df)

Merging datasets

In [20]:
# Combine all data into one long-format DataFrame
long_df = pd.concat(dfs, ignore_index=True)

long_df.head()

,Area_territoriale,Regione,Prov,Comune_ISTAT,Comune_cat,Sez,Comune_amm,Comune_descrizione,Fascia,Zona,...,Stato_prev,Compr_min,Compr_max,Sup_NL_compr,Loc_min,Loc_max,Sup_NL_loc,Unnamed: 21,listing_id,semester
0,NORD-OVEST,PIEMONTE,AL,1006003.0,A2AA,,A182,ALESSANDRIA,B,B1,...,,1110.0,1670.0,L,3.7,5.6,L,NaN,17811240733772113484,2004_S1
1,NORD-OVEST,PIEMONTE,AL,1006003.0,A2AA,,A182,ALESSANDRIA,B,B1,...,,1010.0,1520.0,L,4.3,6.4,L,NaN,3961253180033836184,2004_S1
2,NORD-OVEST,PIEMONTE,AL,1006003.0,A2AA,,A182,ALESSANDRIA,B,B1,...,,610.0,840.0,L,2.5,3.5,L,NaN,2425994923825000307,2004_S1
3,NORD-OVEST,PIEMONTE,AL,1006003.0,A2AA,,A182,ALESSANDRIA,B,B1,...,,490.0,660.0,L,2.0,2.7,L,NaN,15471412781792669566,2004_S1
4,NORD-OVEST,PIEMONTE,AL,1006003.0,A2AA,,A182,ALESSANDRIA,B,B1,...,,1180.0,1410.0,L,5.4,6.5,L,NaN,14067437536608335706,2004_S1


Translating

In [21]:
column_renames = {
    "Area_territoriale": "area",
    "Regione": "region",
    "Prov": "prov",
    "Comune_ISTAT": "mun_istat",
    "Comune_cat": "mun_cat",
    "Comune_amm": "mun_cad",
    "Comune_descrizione": "mun_name",
    "Fascia": "sector",
    "Zona": "zone",
    "Descr_Tipologia": "type",
    "Stato": "condition",
    "Compr_min": "buy_min",
    "Compr_max": "buy_max",
    "Loc_min": "lease_min",
    "Loc_max": "lease_max"
}

long_df = long_df.rename(columns=column_renames)

# Apply value mappings
long_df["area"] = long_df["area"].replace({
    "NORD-OVEST": "NW",
    "NORD-EST": "NE",
    "CENTRO": "C",
    "ISOLE": "I",
    "SUD": "S"
})

long_df["type"] = long_df["type"].replace({
    "Abitazioni civili": "residential housing",
    "Box": "garage",
    "Ville e Villini": "independent houses and villas",
    "Negozi": "shops",
    "Abitazioni di tipo economico": "lowcost housing",
    "Magazzini": "warehouses",
    "Uffici": "offices",
    "Laboratori": "laboratories",
    "Capannoni tipici": "typical industrial buildings",
    "Capannoni industriali": "industrial buildings",
    "Autorimesse": "garages",
    "Posti auto scoperti": "uncovered parking spaces",
    "Posti auto coperti": "covered parking spaces",
    "Centri commerciali": "shopping centers",
    "Uffici strutturati": "structured offices",
    "Abitazioni tipiche dei luoghi": "typical local housing",
    "Abitazioni signorili": "luxury housing",
    "Pensioni e assimilati": "guesthouses and similar",
    "Fabbricati e locali per esercizi sportivi": "sports facilities"
})

long_df["condition"] = long_df["condition"].replace({
    "NORMALE": "normal",
    "OTTIMO": "excellent",
    "SCADENTE": "poor"
})

long_df.head()

,area,region,prov,mun_istat,mun_cat,Sez,mun_cad,mun_name,sector,zone,...,Stato_prev,buy_min,buy_max,Sup_NL_compr,lease_min,lease_max,Sup_NL_loc,Unnamed: 21,listing_id,semester
0,NW,PIEMONTE,AL,1006003.0,A2AA,,A182,ALESSANDRIA,B,B1,...,,1110.0,1670.0,L,3.7,5.6,L,NaN,17811240733772113484,2004_S1
1,NW,PIEMONTE,AL,1006003.0,A2AA,,A182,ALESSANDRIA,B,B1,...,,1010.0,1520.0,L,4.3,6.4,L,NaN,3961253180033836184,2004_S1
2,NW,PIEMONTE,AL,1006003.0,A2AA,,A182,ALESSANDRIA,B,B1,...,,610.0,840.0,L,2.5,3.5,L,NaN,2425994923825000307,2004_S1
3,NW,PIEMONTE,AL,1006003.0,A2AA,,A182,ALESSANDRIA,B,B1,...,,490.0,660.0,L,2.0,2.7,L,NaN,15471412781792669566,2004_S1
4,NW,PIEMONTE,AL,1006003.0,A2AA,,A182,ALESSANDRIA,B,B1,...,,1180.0,1410.0,L,5.4,6.5,L,NaN,14067437536608335706,2004_S1


Deleting useless columns

In [22]:
columns = ['Sez', 'Stato_prev', 'Sup_NL_compr', 'Sup_NL_loc', 'Unnamed: 21']

long_df = long_df.drop(columns=columns)

Ordering variables

In [23]:
new_order = [
    "listing_id", "semester", "area", "region", "prov", "mun_istat", "mun_cat", "mun_cad", 
    "mun_name", "sector", "zone", "type", "condition", "buy_min", "buy_max", "lease_min", "lease_max"
]

long_df = long_df[new_order]

long_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7201300 entries, 0 to 7201299
Data columns (total 17 columns):
 #   Column      Dtype  
---  ------      -----  
 0   listing_id  object 
 1   semester    object 
 2   area        object 
 3   region      object 
 4   prov        object 
 5   mun_istat   float64
 6   mun_cat     object 
 7   mun_cad     object 
 8   mun_name    object 
 9   sector      object 
 10  zone        object 
 11  type        object 
 12  condition   object 
 13  buy_min     float64
 14  buy_max     float64
 15  lease_min   float64
 16  lease_max   float64
dtypes: float64(5), object(12)
memory usage: 934.0+ MB


Checking IDs

In [24]:
# Number of unique listings
print("Number of unique listings:", long_df['listing_id'].nunique())

# Number of duplicate listings for the same semester
duplicates = long_df.duplicated(subset=['listing_id', 'semester'], keep=False)
print("Number of duplicate listings for the same semester:", duplicates.sum())

# Delete duplicate listings for the same semester, keeping the first occurrence
long_df = long_df.drop_duplicates(subset=['listing_id', 'semester'], keep='first')

print('Number of rows after removing duplicates:', len(long_df))

Number of unique listings: 522240
Number of duplicate listings for the same semester: 1430
Number of rows after removing duplicates: 7200542
